In [1]:
import os
import shutil
import glob
import time
from contextlib import redirect_stdout
import traceback

from instancia import parse_instance_fully_correct
from solucao import (
    preparar_servicos,
    construtivo_savings,
    print_solucao_padroes,
)
from grafo import CustomGraphFinal
from otimizacao import swap_entre_rotas

In [ ]:
# Defina o caminho da instância que deseja analisar individualmente
file_path = 'dados/BHW1.dat'

# Leitura e parsing da instância
parsed_data = parse_instance_fully_correct(file_path)

# Criação do grafo
g = CustomGraphFinal(parsed_data)

In [ ]:
# Visualize o grafo
desenhar_grafo(g)

In [ ]:
# Calcule estatísticas e métricas
dist, pred, node_to_index, index_to_node = g.floyd_warshall()
avg_path, diameter = g.average_path_length_and_diameter(dist)
df = pd.DataFrame([
    ("Quantidade de vértices", g.num_vertices()),
    ("Quantidade de arestas", g.num_edges()),
    ("Quantidade de arcos", g.num_arcs()),
    ("Quantidade de vértices requeridos", g.num_required_nodes()),
    ("Quantidade de arestas requeridas", g.num_required_edges()),
    ("Quantidade de arcos requeridos", g.num_required_arcs()),
    ("Densidade do grafo", g.density()),
    ("Componentes conectados", len(g.connected_components())),
    ("Grau mínimo", g.degrees()[0]),
    ("Grau máximo", g.degrees()[1]),
    ("Caminho médio", avg_path),
    ("Diâmetro", diameter)
], columns=["Métrica", "Valor"])
print(df)

# Betweenness centrality
bt_raw = g.betweenness_centrality(dist, pred)
bt = {index_to_node[i]: bt_raw[i] for i in bt_raw}
print("Intermediação:", bt)

In [ ]:
# Preparação dos serviços e IDs
servicos, servico_ids = preparar_servicos(parsed_data, g.depot)

# Geração das soluções (algoritmo construtivo)
solucoes = construtivo_savings(
    g, servicos, servico_ids, g.depot, g.capacity, dist, node_to_index
)

# Impressão da solução no padrão solicitado
print_solucao_padroes(
    solucoes,
    parsed_data["meta"]["name"],
    servico_ids,
    clocks_ref=0,
    clocks_sol=0
)

In [ ]:
from otimizacao import swap_entre_rotas

# ... (código do construtivo)
solucoes = construtivo_savings(...)
solucoes_melhoradas = swap_entre_rotas(
    solucoes, g, servico_ids, dist, node_to_index
)

In [ ]:
print_solucao_padroes(
    solucoes_melhoradas,
    parsed_data["meta"]["name"],
    servico_ids,
    clocks_ref=0,
    clocks_sol=0
)

In [ ]:
INPUT_DIR  = 'dados/'
OUTPUT_DIR = 'solucoes/'

# Limpa a pasta de saída ANTES de começar
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

for filepath in sorted(glob.glob(os.path.join(INPUT_DIR, '*.dat'))):
    inst_name = os.path.splitext(os.path.basename(filepath))[0]
    out_path = os.path.join(OUTPUT_DIR, f'sol-{inst_name}.dat')
    print(f'▶ Processando {inst_name} → {out_path}')
    try:
        with open(out_path, 'w') as fout, redirect_stdout(fout):
            data = parse_instance_fully_correct(filepath)
            g = CustomGraphFinal(data)
            dist, pred, node_to_index, index_to_node = g.floyd_warshall()
            servicos, servico_ids = preparar_servicos(data, g.depot)
            start_alg = time.perf_counter()
            solucoes = construtivo_savings(
                g, servicos, servico_ids, g.depot, g.capacity, dist, node_to_index
            )
            # Aplique a melhoria com swap, passando servicos
            solucoes_melhoradas = swap_entre_rotas(
                solucoes, g, servico_ids, dist, node_to_index, servicos
            )
            end_alg = time.perf_counter()
            clocks_ref = int((end_alg - start_alg) * 1e6)
            print_solucao_padroes(
                solucoes_melhoradas,
                data["meta"]["name"],
                servico_ids,
                clocks_ref=clocks_ref,
                clocks_sol=clocks_ref
            )
        print(f'✅ OK: {out_path}')
    except Exception as e:
        print(f'❌ NÃO GERADO: {inst_name} → {e.__class__.__name__}')
        traceback.print_exc()

▶ Processando BHW1 → solucoes/sol-BHW1.dat
✅ OK: solucoes/sol-BHW1.dat
▶ Processando BHW10 → solucoes/sol-BHW10.dat
✅ OK: solucoes/sol-BHW10.dat
▶ Processando BHW11 → solucoes/sol-BHW11.dat
✅ OK: solucoes/sol-BHW11.dat
▶ Processando BHW12 → solucoes/sol-BHW12.dat
✅ OK: solucoes/sol-BHW12.dat
▶ Processando BHW13 → solucoes/sol-BHW13.dat
✅ OK: solucoes/sol-BHW13.dat
▶ Processando BHW14 → solucoes/sol-BHW14.dat
✅ OK: solucoes/sol-BHW14.dat
▶ Processando BHW15 → solucoes/sol-BHW15.dat
✅ OK: solucoes/sol-BHW15.dat
▶ Processando BHW16 → solucoes/sol-BHW16.dat
